In [ ]:
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

# ========================== Streamlit UI ==========================
st.title("📈 LSTM Stock Forecasting App")
st.markdown("### Intelligent Forecasting using STL + LSTM (Trend + Seasonal)")

# Sidebar inputs
st.sidebar.header("Stock Settings")
ticker = st.sidebar.text_input("Enter Stock Symbol (e.g., BHARTIARTL.NS)", "BHARTIARTL.NS")
start_date = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
end_date_forecast = st.sidebar.date_input("End Date", pd.to_datetime("2025-08-06"))
window_size = st.sidebar.slider("LSTM Window Size", 5, 60, 10)
epochs = st.sidebar.slider("Training Epochs", 10, 200, 50)

if st.sidebar.button("Run Forecast"):
    st.write(f"### 📅 Forecasting for: {ticker}")
    st.write(f"Data from {start_date} to {end_date_forecast}")

    # ========================== Step 1: Download Data ==========================
    data = yf.download(ticker, start=start_date, end=end_date_forecast)
    if data.empty:
        st.error("⚠️ No data found! Please check ticker or date range.")
        st.stop()

    d_high = data['Low']
    st.line_chart(d_high, use_container_width=True)

    # ========================== Step 2: STL Decomposition ==========================
    st.subheader("🔍 STL Decomposition")
    stl = STL(d_high, period=30)
    result = stl.fit()
    trend, seasonal, resid = result.trend, result.seasonal, result.resid

    fig, ax = plt.subplots(3, 1, figsize=(10, 6))
    ax[0].plot(trend); ax[0].set_title("Trend")
    ax[1].plot(seasonal); ax[1].set_title("Seasonal")
    ax[2].plot(resid); ax[2].set_title("Residual")
    st.pyplot(fig)

    # ========================== Step 3: Prepare LSTM Data ==========================
    def prepare_lstm_data(series, window_size):
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(series.values.reshape(-1, 1))
        X, y = [], []
        for i in range(len(scaled) - window_size):
            X.append(scaled[i:i + window_size])
            y.append(scaled[i + window_size])
        return np.array(X), np.array(y), scaler

    X_trend, y_trend, scaler_trend = prepare_lstm_data(trend, window_size)
    X_seasonal, y_seasonal, scaler_seasonal = prepare_lstm_data(seasonal, window_size)

    # ========================== Step 4: Build LSTM ==========================
    def build_and_train_lstm(X, y, epochs):
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
            LSTM(50, return_sequences=False),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X, y, epochs=epochs, batch_size=10, verbose=0)
        return model

    st.info("🧠 Training LSTM Models... (This may take a few seconds)")
    model_trend = build_and_train_lstm(X_trend, y_trend, epochs)
    model_seasonal = build_and_train_lstm(X_seasonal, y_seasonal, epochs)

    # ========================== Step 5: Predictions ==========================
    y_trend_pred = model_trend.predict(X_trend)
    y_seasonal_pred = model_seasonal.predict(X_seasonal)

    trend_pred = scaler_trend.inverse_transform(y_trend_pred)
    seasonal_pred = scaler_seasonal.inverse_transform(y_seasonal_pred)

    final_pred = trend_pred.flatten() + seasonal_pred.flatten()
    actual = d_high.values[window_size:]

    rmse = np.sqrt(mean_squared_error(actual, final_pred))
    st.success(f"📉 Final RMSE (Reconstructed): {rmse:.4f}")

    # ========================== Step 6: Plot Results ==========================
    st.subheader("📊 Actual vs Predicted")
    fig2, ax2 = plt.subplots(figsize=(10, 5))
    ax2.plot(actual, label="Actual", color='blue')
    ax2.plot(final_pred, label="Predicted (Trend + Seasonal)", color='orange')
    ax2.legend(); ax2.grid(True)
    st.pyplot(fig2)

    # ========================== Step 7: Forecast Next Day ==========================
    last_trend_scaled = scaler_trend.transform(trend.values[-window_size:].reshape(-1, 1)).reshape(1, window_size, 1)
    next_trend_scaled = model_trend.predict(last_trend_scaled)
    next_trend = scaler_trend.inverse_transform(next_trend_scaled)[0][0]

    last_seasonal_scaled = scaler_seasonal.transform(seasonal.values[-window_size:].reshape(-1, 1)).reshape(1, window_size, 1)
    next_seasonal_scaled = model_seasonal.predict(last_seasonal_scaled)
    next_seasonal = scaler_seasonal.inverse_transform(next_seasonal_scaled)[0][0]

    next_day_forecast = next_trend + next_seasonal
    st.subheader("🔮 Next Day Forecast")
    st.metric(label="Predicted Low Price for Next Day", value=f"{next_day_forecast:.2f} INR")

    st.caption("Developed with ❤️ using Streamlit and TensorFlow")

else:
    st.info("👈 Enter your stock details and click **Run Forecast** to begin.")
